In [1]:
!which python

/home/sandro/.local/share/virtualenvs/data_science_for_business_lmu-1wV0OWeh/bin/python


# Baseline

In here I create a basic baseline model with just some features and less preprocessing.

In [2]:
def read_in_data(path_bitcoin_df = '../data/raw/1_training_data_sets/1_bitcoin_price_data_set.csv',
                path_training_df = '../data/raw/1_training_data_sets/1_training_data.csv'):
    df_bitcoin = pd.read_csv('../data/raw/1_training_data_sets/1_bitcoin_price_data_set.csv', 
                            encoding = "ISO-8859-1")
    df = pd.read_csv('../data/raw/1_training_data_sets/1_training_data.csv',
                             encoding = "ISO-8859-1")
    
    print("Shape of df_bitcoin: {}".format(df_bitcoin.shape))
    print("Shape of df: {}".format(df.shape))
    return df_bitcoin, df

In [3]:
import pandas as pd
import numpy as np 
from datetime import datetime

In [4]:
df_bitcoin, df = read_in_data()

Shape of df_bitcoin: (2355, 1)
Shape of df: (4757, 91)


Too less -> Pick features first

## Pick basic features

In [5]:
df_features = df[[
    'categories_0',
    'transaction_count',
    'holder_count',
    'timestamp',
    'country_origin'
]]

In [6]:
df_features.head()

,categories_0,transaction_count,holder_count,timestamp,country_origin
0,Masternodes,NaN,NaN,Sep-26-2019 15:59:01 PM,NaN
1,Protocol,975773.0,109840.0,Sep-26-2019 01:56:58 PM,US
2,Protocol,22730.0,13914.0,Sep-25-2019 11:58:48 PM,SI
3,NaN,125.0,51.0,Sep-26-2019 04:58:35 AM,NaN
4,NaN,16417.0,6250.0,Sep-25-2019 11:14:11 PM,NaN


In [7]:
for col in df_features.columns:
    print("Column {} has {} NA values".format(col, df_features[col].isna().sum()))

Column categories_0 has 2609 NA values
Column transaction_count has 2233 NA values
Column holder_count has 2237 NA values
Column timestamp has 1908 NA values
Column country_origin has 2826 NA values


In [8]:
# Categories NA: OHE 
# Transaction count avg
# Holder count avg
# Timestamp remove
# Country origin OHE

## Category 1

In [46]:
def add_category_feature(df, header=None):
    ohe_cat1 = pd.get_dummies(df.categories_0, prefix='categories_1', dummy_na=True)
    ohe_cat1 = pd.concat([df['OBS_ID'], ohe_cat1], axis=1)
    return ohe_cat1

In [52]:
ohe_cat_test = add_category_feature(df_test)

In [54]:
ohe_cat_test

,OBS_ID,categories_1_Artificial Intelligence,categories_1_Asset-backed Tokens,categories_1_Big Data,categories_1_Business Platform,categories_1_Business Services,categories_1_Charity,categories_1_Communication,categories_1_Cryptocurrency,categories_1_Decentralized Exchange,...,categories_1_Retail,categories_1_Smart Contract Platform,categories_1_Software,categories_1_Sports,categories_1_Stablecoins,categories_1_Storage,categories_1_Technology & Science,categories_1_Tourism,categories_1_Virtual Reality,categories_1_nan
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
998,999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
999,1000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
# End df for model
df_X = df['OBS_ID'].copy()

In [10]:
ohe_cat1 = pd.get_dummies(df.categories_0, prefix='categories_1', dummy_na=True)
ohe_cat1 = pd.concat([df['OBS_ID'], ohe_cat1], axis=1) 

In [43]:
df.categories_0

0          Masternodes
1             Protocol
2             Protocol
3                  NaN
4                  NaN
             ...      
4749               NaN
4750    Cryptocurrency
4751    Cryptocurrency
4752               NaN
4755               NaN
Name: categories_0, Length: 2844, dtype: object

In [11]:
ohe_cat1.head()

,OBS_ID,categories_1_Artificial Intelligence,categories_1_Asset-backed Tokens,categories_1_Augmented Reality,categories_1_Big Data,categories_1_Business Platform,categories_1_Business Services,categories_1_Charity,categories_1_Communication,categories_1_Cryptocurrency,...,categories_1_Retail,categories_1_Smart Contract Platform,categories_1_Software,categories_1_Sports,categories_1_Stablecoins,categories_1_Storage,categories_1_Technology & Science,categories_1_Tourism,categories_1_Virtual Reality,categories_1_nan
0,235691,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,235692,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,235693,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,235694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,235695,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
df_X = pd.merge(df_X, ohe_cat1)

In [13]:
df_X.shape

(4757, 41)

## Transaction Count

In [14]:
df.transaction_count.fillna(df.transaction_count.mean(), inplace=True)

In [15]:
df_X = pd.merge(df_X, df[['OBS_ID', 'transaction_count']])

In [16]:
df_X.shape

(4757, 42)

## Holder Count

In [17]:
df.holder_count.fillna(df.holder_count.mean(), inplace=True)

In [18]:
df_X = pd.merge(df_X, df[['OBS_ID', 'holder_count']])

In [19]:
df_X.shape

(4757, 43)

## Timestamp

In [20]:
df = df[~df.timestamp.isna()]

In [21]:
df['timestamp']=df.timestamp.astype(str)

In [22]:
df.shape

(2849, 91)

In [23]:
df = df.assign(timestamp_datetime = df.apply(lambda x:
                                            datetime.strptime(str(x.timestamp), '%b-%d-%Y %H:%M:%S %p'),
                                            axis=1))

In [24]:
df = df.assign(hour = df.apply(lambda x: x.timestamp_datetime.hour, axis=1))
df = df.assign(minute = df.apply(lambda x: x.timestamp_datetime.minute, axis=1))
df = df.assign(second = df.apply(lambda x: x.timestamp_datetime.second, axis=1))
df = df.assign(month = df.apply(lambda x: x.timestamp_datetime.month, axis=1))
df = df.assign(day = df.apply(lambda x: x.timestamp_datetime.day, axis=1))
df = df.assign(year = df.apply(lambda x: x.timestamp_datetime.year, axis=1))

df = df[df.year != 1970]
df = df[df.year != 2017]

In [25]:
df_X = pd.merge(df_X, df[['OBS_ID', 'hour', 'minute', 'second', 'month', 'day', 'year']])

In [26]:
df_X.shape

(2844, 49)

## Country origin

In [27]:
ohe_country = pd.get_dummies(df.country_origin, prefix='country', dummy_na=True)
ohe_country = pd.concat([df['OBS_ID'], ohe_country], axis=1) 

In [28]:
df_X = pd.merge(df_X, ohe_country)

In [29]:
df_X.shape

(2844, 145)

In [30]:
df_X = pd.merge(df_X, df[['OBS_ID', 'success']])

In [31]:
df_X.to_csv('../data/processed/baselince.csv', index=None)

In [32]:
df_X.shape

(2844, 146)

# Training

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
X = df_X.iloc[:, :-1]

In [35]:
y = df_X.iloc[:, -1]

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
clf = RandomForestClassifier(n_estimators=500)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
from sklearn.metrics import matthews_corrcoef

In [41]:
matthews_corrcoef(y_test, clf.predict(X_test))

0.11474387998748653

In [47]:
df_test = pd.read_csv('../data/raw/2_classification_data.csv', encoding = "ISO-8859-1")

In [45]:
df_X.columns

Index(['OBS_ID', 'categories_1_Artificial Intelligence',
       'categories_1_Asset-backed Tokens', 'categories_1_Augmented Reality',
       'categories_1_Big Data', 'categories_1_Business Platform',
       'categories_1_Business Services', 'categories_1_Charity',
       'categories_1_Communication', 'categories_1_Cryptocurrency',
       ...
       'country_VC', 'country_VE', 'country_VG', 'country_VN', 'country_VU',
       'country_YD', 'country_ZA', 'country_ZZ', 'country_nan', 'success'],
      dtype='object', length=146)